In [ ]:
using Pkg
Pkg.activate(".")
# Add the required packages to the environment
Pkg.add(["Images", "Flux", "MLDatasets", "CUDA", "Parameters", "ProgressMeter"])
using Base.Iterators: partition
using Flux
using Flux.Optimise: update!, train!
using Flux.Losses: logitbinarycrossentropy
using Images
using MLDatasets
using Statistics
using Printf
using Random
using CUDA
using ProgressMeter: @showprogress

Base.@kwdef struct HyperParams
    batch_size::Int = 128
    epochs::Int = 20
    verbose_freq::Int = 1000
    lr::Float32 = 0.0002
end

hparams = HyperParams()


In [ ]:

dataset = MLDatasets.MNIST(:train)
dataset_test = MLDatasets.MNIST(:test)

images = dataset.features
labels = dataset.targets

indices = findall(val-> val < 2, labels)
images = images[:, :, indices]
labels = labels[indices]

indices = findall(val-> val < 2, dataset_test.targets)
images_test = dataset_test.features[:, :, indices]
labels_test = dataset_test.targets[indices]

In [ ]:
randinit(shape...) = randn(Float32, shape...)
function Convolutional()
    return Chain(
        Conv((4, 4), 1 => 4, stride=2, pad=1, init=randinit),
        x->leakyrelu.(x, 0.2f0),
        Dropout(0.2),
        Conv((4, 4), 4 => 16, stride=2, pad=1, init=randinit),
        x->leakyrelu.(x, 0.2f0),
        Dropout(0.2),
    )
end
convlayers = Convolutional()
dummy_input = reshape(zeros, 28, 28, 1, :)
dummy_output = convlayers(dummy_input[:, :, :, 1:1])

function EndLayers()
    osize = reduce(*, size(dummy_output)[1:3])
    return Chain(
        x->reshape(x, osize, :),
        Dense(osize, 1),
    )
end

ConvModel = Chain(Convolutional(), EndLayers()) |> gpu

function getloss(model, data, label)
    
    output = model(data)

    one = logitbinarycrossentropy(output[findall(x->x==1, label)], 1)
    zero = logitbinarycrossentropy(output[findall(x->x==0, label)], 0)


    return one + zero
end




In [ ]:


function do_testing(test_data)
    # image_tensor = reshape(@.(2f0 * test_data - 1f0), 28, 28, 1, :)
    correct = 0
    total = 0
    for (data, label) in test_data
        # data = image_tensor[:, :, :, x:x]
        # label = labels_test[x:x]
        inference = ConvModel(data) |> cpu

        for x in 1:size(inference, 2)

            if (inference[1, x] > 0f0 && label == 1f0) || (inference[1, x] < 0 && label == 0)
                correct += 1
            end
            total += 1

        end


        # data = reshape(@.(2f0* images_test[]))
    end
    
    @info "correct: $correct / $total"
end
function do_training()
    image_tensor = reshape(@.(2f0 * images - 1f0), 28, 28, 1, :)
    # label_tensor = reshape(labels, )

    data = [(image_tensor[:, :, :, r] |> gpu, labels[r]) for r in partition(1:size(image_tensor, 4), hparams.batch_size)]
    opt = Flux.setup(Adam(hparams.lr), ConvModel)

    for ep in 1:hparams.epochs
        @info "Epoch $ep"
        train!(getloss, ConvModel, data, opt)
        do_testing(data)
    end
end

In [ ]:
do_training()
